In [1]:
from model import ResidualBlock, Discriminator
import torchvision
import torch

image = torchvision.transforms.Resize(256)(torchvision.io.read_image("face.jpg").float()).unsqueeze(0)

In [2]:
image.size()

torch.Size([1, 3, 256, 256])

In [3]:
rb = ResidualBlock(3, 16)

In [4]:
out = rb(image)
out.size()

torch.Size([1, 16, 128, 128])

In [5]:
d = Discriminator(7, 16)

In [6]:
print(image.size())
d(image)

torch.Size([1, 3, 256, 256])


tensor([[0.0526]], grad_fn=<AddmmBackward0>)

In [7]:
test = torch.zeros((8,3,256,256))
d(test).size()

torch.Size([8, 1])

In [8]:
import torch
torch.manual_seed(9)
#layer = torch.nn.Linear(1, 4)
# layer.weight *= 4 inplace operations on leaf variables does not work

con_layer = torch.nn.Conv2d(4,16, kernel_size=3)
test = torch.zeros((8,4,256,256))
test_unf = torch.nn.functional.unfold(test, (3, 3))
weight = torch.randn(size=(16, 4, 3, 3))
a = test_unf.transpose(1, 2).matmul(weight.view(weight.size(0), -1).t())
print(a.size())
print(con_layer.weight.size(), con_layer(test).size(), test.size(), test_unf.size())

torch.Size([8, 64516, 16])
torch.Size([16, 4, 3, 3]) torch.Size([8, 16, 254, 254]) torch.Size([8, 4, 256, 256]) torch.Size([8, 36, 64516])


In [9]:
import torch
from torch import nn
unfold = nn.Unfold(kernel_size=(2, 3))
input = torch.randn(2, 5, 3, 4)
output = unfold(input)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
# print(input.size(),output.size())

# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
inp = torch.randn(4, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
print(inp_unf.size())
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
print(out_unf.size())
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
print(out.size())
# or equivalently (and avoiding a copy),
# out = out_unf.view(1, 2, 7, 8)
(torch.nn.functional.conv2d(inp, w) - out).abs().max()

torch.Size([4, 60, 56])
torch.Size([4, 2, 56])
torch.Size([4, 2, 7, 8])


tensor(5.7220e-06)

In [10]:
w.view(w.size(0), -1).t().size()

torch.Size([60, 2])

In [51]:
torch.manual_seed(9)
def convolution(i, w):

    inp_unf = torch.nn.functional.unfold(i, (w.size(2), w.size(3)))
    if len(w.size()) == 4:
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
    else:
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), w.size(1), -1).transpose(1,2)).transpose(1, 2)
    out = torch.nn.functional.fold(out_unf, (i.size(2)-w.size(2)+1, i.size(3)-w.size(3)+1), (1, 1))
    print(out.size())
    return out

In [53]:
torch.manual_seed(9)
i = torch.ones((8,3,6,6))
s2 = torch.rand(8,1,3,1,1)
s1 = s2.view(8,3,1,1)
# print(s)
w = torch.ones((16, 3, 3, 3))
o1 = convolution(i*s1, w)
bias = torch.randn((16,1,1))
print(o1+bias)
o2 = convolution(i, s2*w)
print((~(torch.abs(o1-o2) <1e-3)).sum())

torch.Size([8, 16, 4, 4])
tensor([[[[13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770]],

         [[13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996]],

         [[12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039]],

         ...,

         [[12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737]],

         [[11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 1

In [14]:
a = torch.randn((8,3,2,1,1))
a.square().sum(dim=(2,3,4), keepdim=True).size()

torch.Size([8, 3, 1, 1, 1])

In [16]:
a,a.square().sum(dim=(2,3,4))

(tensor([[[[[ 0.9826]],
 
           [[ 1.3936]]],
 
 
          [[[-0.5419]],
 
           [[-0.4245]]],
 
 
          [[[ 0.4435]],
 
           [[-0.5365]]]],
 
 
 
         [[[[-1.4623]],
 
           [[-0.8552]]],
 
 
          [[[ 1.7615]],
 
           [[ 0.0027]]],
 
 
          [[[ 1.0803]],
 
           [[-1.9376]]]],
 
 
 
         [[[[-0.5748]],
 
           [[ 0.4349]]],
 
 
          [[[-1.6585]],
 
           [[ 1.2364]]],
 
 
          [[[ 1.2487]],
 
           [[-0.7758]]]],
 
 
 
         [[[[-0.6506]],
 
           [[ 1.0455]]],
 
 
          [[[ 0.9524]],
 
           [[ 1.4272]]],
 
 
          [[[ 0.2012]],
 
           [[-0.9011]]]],
 
 
 
         [[[[-0.1297]],
 
           [[ 1.0740]]],
 
 
          [[[-0.7193]],
 
           [[ 1.0961]]],
 
 
          [[[-0.1835]],
 
           [[ 0.7050]]]],
 
 
 
         [[[[-1.0667]],
 
           [[ 0.2605]]],
 
 
          [[[-0.5587]],
 
           [[-0.7631]]],
 
 
          [[[ 0.9615]],
 
           [[-0.4455]]]]

In [17]:
from model import ModDemodConv3x3
mdconv = ModDemodConv3x3(3, 16)
i = torch.ones((8,3,256,256))
s = torch.rand(8,3)
o=mdconv(i,s)
o.size()

torch.Size([8, 16, 3, 3, 3])
torch.Size([8, 16, 1, 1, 1])
torch.Size([8, 16, 3, 3, 3])


torch.Size([8, 16, 256, 256])

In [10]:
input = torch.arange(1, 5, dtype=torch.float32).view(1, 1, 2, 2)
print(input.size())

m = torch.nn.Upsample(scale_factor=2, mode='nearest')
print(m(input).size())

m = torch.nn.Upsample(scale_factor=2, mode='bilinear')  
print(m(input).size())

torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 4])


In [15]:
x = torch.tensor([[1], [2], [3]])
print(x.size())
print(x.repeat(1, 4).size())   # -1 means not changing the size of that dimension
print(x, x.repeat(1,4))

torch.Size([3, 1])
torch.Size([3, 4])
tensor([[1],
        [2],
        [3]]) tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])


In [17]:
x.unsqueeze(0).size()

torch.Size([1, 3, 1])

In [18]:
torch.zeros([]).size()

torch.Size([])

In [28]:
noise_scaling_factor = torch.tensor([2])
noise = torch.randn((1,1,4,4))

In [30]:
noise, noise_scaling_factor * noise, noise_scaling_factor.size()

(tensor([[[[ 0.5647, -1.3814, -1.1341,  0.3085],
           [-0.7618,  0.3484,  1.7113,  1.5437],
           [-0.7767, -0.6729, -0.3932,  0.7823],
           [-0.5935,  0.7784, -0.1440,  0.5170]]]]),
 tensor([[[[ 1.1293, -2.7628, -2.2682,  0.6170],
           [-1.5236,  0.6969,  3.4226,  3.0875],
           [-1.5533, -1.3458, -0.7864,  1.5645],
           [-1.1869,  1.5568, -0.2881,  1.0340]]]]),
 torch.Size([1]))

In [32]:
torch.zeros((1))*noise

tensor([[[[0., -0., -0., 0.],
          [-0., 0., 0., 0.],
          [-0., -0., -0., 0.],
          [-0., 0., -0., 0.]]]])

In [39]:
upsample = torch.nn.Upsample(scale_factor=2, mode="bilinear")
inp = torch.randn(8,3,4,4)
inp.size(), upsample(inp).size()

(torch.Size([8, 3, 4, 4]), torch.Size([8, 3, 8, 8]))

In [44]:
layer = torch.nn.Conv2d(3,16,3)
layer.weight.size(), layer.bias.size()

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [45]:
inp = torch.randn((8,3,16,16))

In [46]:
layer(inp).size()

torch.Size([8, 16, 14, 14])